In [1]:
import cv2
import numpy as np
import xml.dom.minidom
import tensorflow as tf
import os
import time
from tensorflow.python.framework import graph_util

slim = tf.contrib.slim

/data/liuan/miniconda2/envs/tf/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/data/liuan/miniconda2/envs/tf/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/data/liuan/miniconda2/envs/tf/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/data/liuan/miniconda2/envs/tf/lib/python3.6/site-packages

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
#读取txt文件
train_txt = open('/data/liuan/jupyter/root/project/keras-retinanet-master/bbox_fz_zc/dataset/txtsave/train.txt')   
val_txt = open('/data/liuan/jupyter/root/project/keras-retinanet-master/bbox_fz_zc/dataset/txtsave/val.txt')
train_content = train_txt.readlines()   #保存的train.txt中的内容
val_content = val_txt.readlines()  #保存的val.txt中的内容
for linetr in train_content:
    print (linetr.rstrip('\n'))
for lineva in val_content:
    print (lineva.rstrip('\n'))

000838_fz
000690_fz
000676_fz
000941_fz
000632_fz
000924_fz
001006_zc
001087_zc
000745_fz
000826_fz
000619_fz
000702_fz
000973_fz
000612_fz
000861_fz
000959_zc
000784_fz
001033_fz
000967_fz
001063_zc
000854_fz
000679_fz
001072_zc
000827_zc
000766_fz
000941_zc
001028_zc
000888_zc
001006_fz
000619_zc
001016_fz
000791_fz
000643-Copy1_zc
001044_fz
000774_zc
000664_zc
000791_zc
000905_zc
000869_zc
000851_fz
000784_zc
000726_fz
000663_fz
001048_zc
000738_zc
000778_fz
000704_fz
001042_zc
000730_zc
000894_fz
000717_fz
000600-Copy1_fz
000864_zc
001063_fz
001022_zc
000936_fz
000949_zc
000601_zc
001013_zc
000849_fz
001080_zc
001015_fz
000688_zc
000980_fz
000715_zc
000957_fz
000730_fz
001003_zc
001055_fz
001085_fz
000620_fz
000612_zc
000610_fz
000627_fz
000602-Copy1_fz
000786_fz
000938_fz
000777_zc
001003_fz
000702_zc
000956_fz
000616_fz
000639_fz
000721_zc
000729_fz
000865_fz
000927_zc
000631_fz
001024_zc
000893_fz
000820_fz
001030_zc
000636_zc
001056_fz
001057_fz
000695_zc
000664_fz
000618_fz
00

In [3]:
#根据txt文件读取图像数据,并归一化图像，并保存缩放比例
train_imgs=[]
train_imgs_ratio=[] #width 缩放比，height缩放比
val_imgs=[]
val_imgs_ratio=[]


# h=48
# w=192  #归一化的尺寸
# c=3   #通道
h=40
w=200
c=3


for linetr in train_content:
    img_path='/data/liuan/jupyter/root/project/keras-retinanet-master/bbox_fz_zc/dataset/cut_any_result/'+linetr.rstrip('\n')+'.jpg'
    img = cv2.imread(img_path)  #读取原图
    print("image_name", str(linetr.rstrip('\n')))
    print("imgshape", img.shape)
    imgresize= cv2.resize(img,(w,h)) #图像归一化
    ratio = np.array([imgresize.shape[0]/img.shape[0], imgresize.shape[1]/img.shape[1]],np.float32) #height缩放比 ,width 缩放比，
    train_imgs_ratio.append(ratio)    
    train_imgs.append(imgresize)
train_img_arr = np.asarray(train_imgs,np.float32)  #保存训练图像数据的列表  h w c
print(len(train_img_arr),len(train_imgs_ratio))

for  lineva in val_content:
    img_path='/data/liuan/jupyter/root/project/keras-retinanet-master/bbox_fz_zc/dataset/cut_any_result/'+lineva.rstrip('\n')+'.jpg'
    img = cv2.imread(img_path) # h w c
    imgresize= cv2.resize(img,(w,h))  #h w c
    ratio = np.array([imgresize.shape[0]/img.shape[0], imgresize.shape[1]/img.shape[1]],np.float32) #height缩放比, width 缩放比，
    val_imgs_ratio.append(ratio)
    val_imgs.append(imgresize)
   # print(imgresize.shape[0], imgresize.shape[1], imgresize.shape[2])
val_img_arr = np.asarray(val_imgs,np.float32)  #保存验证图像的数据的列表 h w c

print(len(val_img_arr),len(val_imgs_ratio))


image_name 000838_fz
imgshape (54, 179, 3)
image_name 000690_fz
imgshape (45, 174, 3)
image_name 000676_fz
imgshape (63, 221, 3)
image_name 000941_fz
imgshape (46, 169, 3)
image_name 000632_fz
imgshape (22, 94, 3)
image_name 000924_fz
imgshape (44, 177, 3)
image_name 001006_zc
imgshape (43, 158, 3)
image_name 001087_zc
imgshape (58, 191, 3)
image_name 000745_fz
imgshape (48, 203, 3)
image_name 000826_fz
imgshape (39, 138, 3)
image_name 000619_fz
imgshape (54, 195, 3)
image_name 000702_fz
imgshape (28, 104, 3)
image_name 000973_fz
imgshape (76, 278, 3)
image_name 000612_fz
imgshape (63, 245, 3)
image_name 000861_fz
imgshape (39, 138, 3)
image_name 000959_zc
imgshape (74, 269, 3)
image_name 000784_fz
imgshape (32, 122, 3)
image_name 001033_fz
imgshape (33, 117, 3)
image_name 000967_fz
imgshape (64, 216, 3)
image_name 001063_zc
imgshape (72, 250, 3)
image_name 000854_fz
imgshape (34, 127, 3)
image_name 000679_fz
imgshape (22, 86, 3)
image_name 001072_zc
imgshape (100, 352, 3)
image_name 0

image_name 000804_zc
imgshape (53, 193, 3)
image_name 001009_fz
imgshape (44, 172, 3)
image_name 000720_fz
imgshape (42, 164, 3)
image_name 000899_zc
imgshape (56, 210, 3)
image_name 001088_fz
imgshape (62, 231, 3)
image_name 000726_zc
imgshape (39, 108, 3)
image_name 000636_fz
imgshape (31, 104, 3)
image_name 000697_fz
imgshape (41, 174, 3)
image_name 000920_fz
imgshape (42, 153, 3)
image_name 000995_fz
imgshape (65, 231, 3)
image_name 000938_zc
imgshape (69, 258, 3)
image_name 000955_fz
imgshape (46, 148, 3)
image_name 000663_zc
imgshape (51, 189, 3)
image_name 001035_zc
imgshape (27, 92, 3)
image_name 000625_fz
imgshape (44, 167, 3)
image_name 000850_fz
imgshape (32, 120, 3)
image_name 000900_fz
imgshape (69, 216, 3)
image_name 000741_fz
imgshape (69, 231, 3)
image_name 000879_zc
imgshape (87, 299, 3)
image_name 000952_fz
imgshape (57, 214, 3)
image_name 000966_fz
imgshape (34, 120, 3)
image_name 000861_zc
imgshape (38, 133, 3)
image_name 000707_zc
imgshape (50, 168, 3)
image_name 0

image_name 000638_zc
imgshape (144, 363, 3)
image_name 000660_zc
imgshape (51, 170, 3)
image_name 000684_zc
imgshape (38, 135, 3)
image_name 000782_zc
imgshape (75, 230, 3)
image_name 000777_fz
imgshape (26, 110, 3)
image_name 000676_zc
imgshape (62, 204, 3)
image_name 000831_fz
imgshape (32, 104, 3)
image_name 000961_zc
imgshape (56, 202, 3)
image_name 000790_zc
imgshape (29, 108, 3)
image_name 000952_zc
imgshape (56, 200, 3)
image_name 000604_fz
imgshape (69, 273, 3)
image_name 000600_zc
imgshape (82, 288, 3)
image_name 000985_fz
imgshape (28, 96, 3)
image_name 000796_zc
imgshape (75, 262, 3)
image_name 001085_zc
imgshape (55, 189, 3)
image_name 000698_fz
imgshape (56, 190, 3)
image_name 000773_fz
imgshape (52, 188, 3)
image_name 000996_zc
imgshape (38, 124, 3)
image_name 000904_zc
imgshape (45, 150, 3)
image_name 000822_fz
imgshape (34, 133, 3)
image_name 000971_fz
imgshape (41, 151, 3)
image_name 001019_fz
imgshape (58, 214, 3)
image_name 000859_zc
imgshape (48, 181, 3)
image_name 

In [10]:
#根据txt文件读取xml,并获取xml中的坐标（xmin,ymin,xmax,ymax）(x表示width,y表示height),并获取经过缩放后的坐标
train_xml=[]  #保存标记的边框坐标
train_xml_resize=[]  #保存标记的边框坐标经过缩放后的坐标，缩放比与图像归一化的缩放比
val_xml=[]
val_xml_resize=[]
for linetr in train_content:
    xml_path='/data/liuan/jupyter/root/project/keras-retinanet-master/bbox_fz_zc/dataset/xml_zc_fz/'+linetr.rstrip('\n')+'.xml'
    print(xml_path)
    xml_DomTree = xml.dom.minidom.parse(xml_path)
    xml_annotation = xml_DomTree.documentElement
    xml_object = xml_annotation.getElementsByTagName('object')
    xml_bndbox = xml_object[0].getElementsByTagName('bndbox')
    xmin_list = xml_bndbox[0].getElementsByTagName('xmin')
    xmin = int(xmin_list[0].childNodes[0].data)
    ymin_list = xml_bndbox[0].getElementsByTagName('ymin')
    ymin = int(ymin_list[0].childNodes[0].data)
    xmax_list = xml_bndbox[0].getElementsByTagName('xmax')
    xmax = int(xmax_list[0].childNodes[0].data)
    ymax_list = xml_bndbox[0].getElementsByTagName('ymax')
    ymax = int(ymax_list[0].childNodes[0].data)
    coordinate = np.array([ymin, xmin ,ymax, xmax],np.int)  #h w h w
    train_xml.append(coordinate) #保存训练图像的xml的坐标
print(len( train_xml))

for lineva in val_content:
    xml_path='/data/liuan/jupyter/root/project/keras-retinanet-master/bbox_fz_zc/dataset/xml_zc_fz/'+lineva.rstrip('\n')+'.xml'
    print(xml_path)
    xml_DomTree = xml.dom.minidom.parse(xml_path)
    xml_annotation = xml_DomTree.documentElement
    xml_object = xml_annotation.getElementsByTagName('object')
    xml_bndbox = xml_object[0].getElementsByTagName('bndbox')
    xmin_list = xml_bndbox[0].getElementsByTagName('xmin')
    xmin = int(xmin_list[0].childNodes[0].data)
    ymin_list = xml_bndbox[0].getElementsByTagName('ymin')
    ymin = int(ymin_list[0].childNodes[0].data)
    xmax_list = xml_bndbox[0].getElementsByTagName('xmax')
    xmax = int(xmax_list[0].childNodes[0].data)
    ymax_list = xml_bndbox[0].getElementsByTagName('ymax')
    ymax = int(ymax_list[0].childNodes[0].data)
    coordinate = np.array([ymin, xmin ,ymax, xmax],np.int)
    val_xml.append(coordinate)  #保存验证图像的xml的坐标
print(len( val_xml))   

for i in range(0,len(train_imgs_ratio)):
    ymin_ratio=train_xml[i][0]*train_imgs_ratio[i][0]
    xmin_ratio=train_xml[i][1]*train_imgs_ratio[i][1]
    ymax_ratio=train_xml[i][2]*train_imgs_ratio[i][0]
    xmax_ratio=train_xml[i][3]*train_imgs_ratio[i][1]
    coordinate_ratio = np.array([ymin_ratio, xmin_ratio ,ymax_ratio, xmax_ratio],np.float32)
    train_xml_resize.append(coordinate_ratio)  #保存训练图像的标记的xml的缩放后的坐标
    
for i in range(0,len(val_imgs_ratio)):
    ymin_ratio=val_xml[i][0]*val_imgs_ratio[i][0]
    xmin_ratio=val_xml[i][1]*val_imgs_ratio[i][1]
    ymax_ratio=val_xml[i][2]*val_imgs_ratio[i][0]
    xmax_ratio=val_xml[i][3]*val_imgs_ratio[i][1]
    coordinate_ratio = np.array([ymin_ratio, xmin_ratio ,ymax_ratio, xmax_ratio],np.float32)
    val_xml_resize.append(coordinate_ratio)   #保存训练验证图像的标记的xml的缩放后的坐标
   


/data/liuan/jupyter/root/project/keras-retinanet-master/bbox_fz_zc/dataset/xml_zc_fz/000838_fz.xml
/data/liuan/jupyter/root/project/keras-retinanet-master/bbox_fz_zc/dataset/xml_zc_fz/000690_fz.xml
/data/liuan/jupyter/root/project/keras-retinanet-master/bbox_fz_zc/dataset/xml_zc_fz/000676_fz.xml
/data/liuan/jupyter/root/project/keras-retinanet-master/bbox_fz_zc/dataset/xml_zc_fz/000941_fz.xml
/data/liuan/jupyter/root/project/keras-retinanet-master/bbox_fz_zc/dataset/xml_zc_fz/000632_fz.xml
/data/liuan/jupyter/root/project/keras-retinanet-master/bbox_fz_zc/dataset/xml_zc_fz/000924_fz.xml
/data/liuan/jupyter/root/project/keras-retinanet-master/bbox_fz_zc/dataset/xml_zc_fz/001006_zc.xml
/data/liuan/jupyter/root/project/keras-retinanet-master/bbox_fz_zc/dataset/xml_zc_fz/001087_zc.xml
/data/liuan/jupyter/root/project/keras-retinanet-master/bbox_fz_zc/dataset/xml_zc_fz/000745_fz.xml
/data/liuan/jupyter/root/project/keras-retinanet-master/bbox_fz_zc/dataset/xml_zc_fz/000826_fz.xml
/data/liua

/data/liuan/jupyter/root/project/keras-retinanet-master/bbox_fz_zc/dataset/xml_zc_fz/000762_zc.xml
/data/liuan/jupyter/root/project/keras-retinanet-master/bbox_fz_zc/dataset/xml_zc_fz/000747_zc.xml
/data/liuan/jupyter/root/project/keras-retinanet-master/bbox_fz_zc/dataset/xml_zc_fz/000970_zc.xml
/data/liuan/jupyter/root/project/keras-retinanet-master/bbox_fz_zc/dataset/xml_zc_fz/000982_fz.xml
/data/liuan/jupyter/root/project/keras-retinanet-master/bbox_fz_zc/dataset/xml_zc_fz/001044_zc.xml
/data/liuan/jupyter/root/project/keras-retinanet-master/bbox_fz_zc/dataset/xml_zc_fz/000712_zc.xml
/data/liuan/jupyter/root/project/keras-retinanet-master/bbox_fz_zc/dataset/xml_zc_fz/000606_zc.xml
/data/liuan/jupyter/root/project/keras-retinanet-master/bbox_fz_zc/dataset/xml_zc_fz/001048_fz.xml
/data/liuan/jupyter/root/project/keras-retinanet-master/bbox_fz_zc/dataset/xml_zc_fz/000794_fz.xml
/data/liuan/jupyter/root/project/keras-retinanet-master/bbox_fz_zc/dataset/xml_zc_fz/000902_fz.xml
/data/liua

In [4]:
#按批次取数据，获取batchsize数据
# inputs 图像数据  归一化后的数据
# targets xml坐标数据  归一化后的数据
def getbatches(inputs=None, targets=None, batch_size=None, shuffle=False):    
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batch_size + 1, batch_size):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batch_size]
        else:
            excerpt = slice(start_idx, start_idx + batch_size)
        yield inputs[excerpt], targets[excerpt]


In [5]:
#损失函数
def abs_smooth(x):
    """Smoothed absolute function. Useful to compute an L1 smooth error.

    Define as:
        x^2 / 2         if abs(x) < 1
        abs(x) - 0.5    if abs(x) > 1
    We use here a differentiable definition using min(x) and abs(x). Clearly
    not optimal, but good enough for our purpose!
    """
    absx = tf.abs(x)
    minx = tf.minimum(absx, 1)
    r = 0.5 * ((absx - 1) * minx + absx)
    return r

In [6]:
#构建网络结构

input_data = tf.placeholder(tf.float32,shape=[None,h,w,c],name='x')  #输入的图像数据（归一化后的图像数据）
input_bound = tf.placeholder(tf.float32,shape=[None,None],name='y') #输入的标记的边框坐标数据（缩放后的xml坐标）
prob=tf.placeholder(tf.float32, name='keep_prob')


#第一个卷积层（192——>96) （48--》24）
#conv1 = slim.repeat(input_data, 2, slim.conv2d, 32, [3, 3], scope='conv1')
conv1 = slim.conv2d(input_data,  32, [3, 3], scope='conv1')
pool1 = slim.max_pool2d(conv1, [2, 2], scope='pool1')

#第二个卷积层（96-48） （24-》12）
#conv2 =  slim.repeat(pool1, 2, slim.conv2d, 64, [3, 3], scope='conv2')
conv2 = slim.conv2d(pool1, 64, [3, 3], scope='conv2')
pool2 = slim.max_pool2d(conv2, [2, 2], scope='pool2')

#第三个卷积层（48-24） （12-》6）
#conv3 = slim.repeat(pool2, 2, slim.conv2d, 128, [3, 3], scope='conv3')
conv3 = slim.conv2d(pool2, 128, [3, 3], scope='conv3')
pool3 = slim.max_pool2d(conv3, [2, 2], scope='pool3')

#第四个卷积层（24） （6）
conv4 = slim.conv2d(pool3, 256 ,[3, 3], scope='conv4')
dropout = tf.layers.dropout(conv4, rate=prob, training=True)
#dropout = tf.nn.dropout(conv4,keep_prob) 
#pool4 = slim.max_pool2d(conv4, [2, 2], scope='pool4')

#第五个卷积层（24-12） （6-》3）
#conv5 = slim.repeat(dropout, 2, slim.conv2d, 128, [3, 3], scope='conv5')
conv5 = slim.conv2d(dropout , 128, [3, 3], scope='conv5')
pool5 = slim.max_pool2d(conv5, [2, 2], scope='pool5')

#第六个卷积层（12-6） （3-》1）
#conv6 = slim.repeat(pool5, 2, slim.conv2d, 64, [3, 3], scope='conv6')
conv6 = slim.conv2d(pool5, 64, [3, 3], scope='conv6')
pool6 = slim.max_pool2d(conv6, [2, 2], scope='pool6')

reshape = tf.reshape(pool6, [-1, 6 * 1 * 64])
print(reshape.get_shape())

fc = slim.fully_connected(reshape, 4, scope='fc')  
print(fc)
print(input_data)

'''
#第七个卷积层（6-3） （1-》1）
conv7 = slim.conv2d(pool6,  32, [3, 3], scope='conv7')
pool7 = slim.max_pool2d(conv7, [2, 2], scope='pool7')

conv8 = slim.conv2d(pool7, 4, [3, 3], padding=None, activation_fn=None,scope='conv8')
'''




Instructions for updating:
Use keras.layers.dropout instead.


(?, 384)
Tensor("fc/Relu:0", shape=(?, 4), dtype=float32)
Tensor("x:0", shape=(?, 40, 200, 3), dtype=float32)


"\n#第七个卷积层（6-3） （1-》1）\nconv7 = slim.conv2d(pool6,  32, [3, 3], scope='conv7')\npool7 = slim.max_pool2d(conv7, [2, 2], scope='pool7')\n\nconv8 = slim.conv2d(pool7, 4, [3, 3], padding=None, activation_fn=None,scope='conv8')\n"

In [8]:
n_epoch =10000
batch_size= 16
print (batch_size)

16


In [13]:
weights = tf.expand_dims(1. * 1., axis=-1)
loss = abs_smooth(fc - input_bound)
print(loss)
train_op=tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

#correct_prediction = tf.equal(fc, input_bound)
#correct_prediction = tf.equal(tf.cast(fc,tf.int32), tf.cast(input_bound, tf.int32))

temp_acc = tf.abs(tf.cast(fc,tf.int32) - tf.cast(input_bound, tf.int32))
compare_np = np.ones((batch_size,4), np.int32)
compare_np[:] = 3
print(compare_np)
compare_tf = tf.convert_to_tensor(compare_np)
print(compare_tf)
correct_prediction = tf.less(temp_acc,compare_tf)
print(correct_prediction)
loss = tf.div(tf.reduce_sum(loss * weights), batch_size, name='value')
tf.summary.scalar('loss',loss) #可视化观看常量  
print(loss)
accuracy= tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#tf.summary.scalar('accuracy',accuracy) #可视化观看常量  
print(accuracy)

Tensor("mul_4:0", shape=(?, 4), dtype=float32)
[[3 3 3 3]
 [3 3 3 3]
 [3 3 3 3]
 [3 3 3 3]
 [3 3 3 3]
 [3 3 3 3]
 [3 3 3 3]
 [3 3 3 3]
 [3 3 3 3]
 [3 3 3 3]
 [3 3 3 3]
 [3 3 3 3]
 [3 3 3 3]
 [3 3 3 3]
 [3 3 3 3]
 [3 3 3 3]]
Tensor("Const_3:0", shape=(16, 4), dtype=int32)
Tensor("Less_1:0", shape=(16, 4), dtype=bool)
Tensor("value_1:0", shape=(), dtype=float32)
Tensor("Mean_1:0", shape=(), dtype=float32)


In [14]:
print(prob)

pb_file_path = '/data/liuan/jupyter/root/project/keras-retinanet-master/bbox_fz_zc/bbox_pb_model/ocr_bboxregress_batch16_epoch10000.pb'


# 设置可见GPU
gpu_no = '3' # or '1'
os.environ["CUDA_VISIBLE_DEVICES"] = gpu_no
#定义TensorFlow配置
config = tf.ConfigProto()
#配置GPU内存分配方式
config.gpu_options.allow_growth = True
# config.gpu_options.per_process_gpu_memory_fraction = 0.4
config.gpu_options.per_process_gpu_memory_fraction = 0.8


sess=tf.InteractiveSession(config = config)  

#////////////////////////////////
#ckpt = tf.train.get_checkpoint_state('/home/data/wangchongjin/ad_image/model_save/')
#saver = tf.train.import_meta_graph(ckpt.model_checkpoint_path +'.meta')   # 载入图结构，保存在.meta文件中
#saver.restore(sess,ckpt.model_checkpoint_path)  
#//////////////////////////////////
sess.run(tf.global_variables_initializer())

merged = tf.summary.merge_all()
writer = tf.summary.FileWriter("/data/liuan/jupyter/root/project/keras-retinanet-master/bbox_fz_zc/record_graph", sess.graph_def)

#saver = tf.train.Saver() # 声明tf.train.Saver类用于保存模型


for epoch in range(n_epoch):
    start_time = time.time()
    
    #training
    train_loss, train_acc, n_batch = 0, 0, 0
    for x_train_a, y_train_a in getbatches(train_img_arr, train_xml_resize, batch_size, shuffle=False):
        _,err,acc=sess.run([train_op,loss,accuracy], feed_dict={input_data: x_train_a, input_bound: y_train_a, prob: 0.5})
        train_loss += err
        train_acc += acc
        n_batch += 1
        
    print(epoch)
    print("   train loss: %f" % (train_loss/ n_batch))
    print("   train acc: %f" % (train_acc/ n_batch))
  


    #validation
    val_loss, val_acc, n_batch = 0, 0, 0
    for x_val_a, y_val_a in getbatches(val_img_arr, val_xml_resize, batch_size, shuffle=False):
        err,acc = sess.run([loss,accuracy], feed_dict={input_data: x_val_a, input_bound: y_val_a, prob: 0})
        #print(err)
        val_loss += err
        val_acc += acc
        n_batch += 1

        rs =sess.run([merged], feed_dict={input_data: x_val_a, input_bound: y_val_a, prob: 0})
        if n_batch is batch_size:
            writer.add_summary(rs[0],epoch)
   
    print("   validation loss: %f" % (val_loss/ n_batch))
    print("   validation acc: %f" % (val_acc/ n_batch))
    
    
#    saver.save(sess, "/home/data/wangchongjin/ad_image/model_save_new/ad.ckpt")
    constant_graph = graph_util.convert_variables_to_constants(sess, sess.graph_def, ['fc/Relu'])

    with tf.gfile.FastGFile(pb_file_path, mode='wb') as f:    
        f.write(constant_graph.SerializeToString())

writer.close()
sess.close()


Tensor("keep_prob:0", dtype=float32)


/data/liuan/miniconda2/envs/tf/lib/python3.6/site-packages/tensorflow/python/client/session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


0
   train loss: 110.607907
   train acc: 0.038603
   validation loss: 105.292587
   validation acc: 0.020833
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 14 variables.
INFO:tensorflow:Converted 14 variables to const ops.
Instructions for updating:
Use tf.gfile.GFile.
1
   train loss: 91.270431
   train acc: 0.040441
   validation loss: 106.469978
   validation acc: 0.026042
INFO:tensorflow:Froze 14 variables.
INFO:tensorflow:Converted 14 variables to const ops.
2
   train loss: 90.271603
   train acc: 0.039062
   validation loss: 88.415794
   validation acc: 0.036458
INFO:tensorflow:Froze 14 variables.
INFO:tensorflow:Converted 14 variables to const ops.
3
   train loss: 94.178073
   train acc: 0.039982
   validation loss: 106.117714
   validation acc: 0.026042
INFO:tensorflow:Froze 14 variables.
INFO:tensorflow:Converted 14 variables to const ops

INFO:tensorflow:Converted 14 variables to const ops.
38
   train loss: 71.591896
   train acc: 0.059283
   validation loss: 74.573362
   validation acc: 0.072917
INFO:tensorflow:Froze 14 variables.
INFO:tensorflow:Converted 14 variables to const ops.
39
   train loss: 69.370571
   train acc: 0.064798
   validation loss: 77.779190
   validation acc: 0.031250
INFO:tensorflow:Froze 14 variables.
INFO:tensorflow:Converted 14 variables to const ops.
40
   train loss: 70.066053
   train acc: 0.064798
   validation loss: 82.794154
   validation acc: 0.020833
INFO:tensorflow:Froze 14 variables.
INFO:tensorflow:Converted 14 variables to const ops.
41
   train loss: 69.938974
   train acc: 0.057904
   validation loss: 78.678406
   validation acc: 0.046875
INFO:tensorflow:Froze 14 variables.
INFO:tensorflow:Converted 14 variables to const ops.
42
   train loss: 68.590145
   train acc: 0.069393
   validation loss: 76.227392
   validation acc: 0.041667
INFO:tensorflow:Froze 14 variables.
INFO:tenso

INFO:tensorflow:Converted 14 variables to const ops.
80
   train loss: 64.087318
   train acc: 0.103401
   validation loss: 68.324855
   validation acc: 0.093750
INFO:tensorflow:Froze 14 variables.
INFO:tensorflow:Converted 14 variables to const ops.
81
   train loss: 63.918428
   train acc: 0.115809
   validation loss: 68.690079
   validation acc: 0.098958
INFO:tensorflow:Froze 14 variables.
INFO:tensorflow:Converted 14 variables to const ops.
82
   train loss: 63.878343
   train acc: 0.110294
   validation loss: 69.637161
   validation acc: 0.093750
INFO:tensorflow:Froze 14 variables.
INFO:tensorflow:Converted 14 variables to const ops.
83
   train loss: 63.416421
   train acc: 0.112592
   validation loss: 68.753782
   validation acc: 0.104167
INFO:tensorflow:Froze 14 variables.
INFO:tensorflow:Converted 14 variables to const ops.
84
   train loss: 63.009935
   train acc: 0.130055
   validation loss: 68.537280
   validation acc: 0.093750
INFO:tensorflow:Froze 14 variables.
INFO:tenso

...

9808
   train loss: 14.711060
   train acc: 0.510110
   validation loss: 23.319185
   validation acc: 0.343750
INFO:tensorflow:Froze 14 variables.
INFO:tensorflow:Converted 14 variables to const ops.
9809
   train loss: 14.510467
   train acc: 0.520221
   validation loss: 24.150389
   validation acc: 0.333333
INFO:tensorflow:Froze 14 variables.
INFO:tensorflow:Converted 14 variables to const ops.
9810
   train loss: 14.332288
   train acc: 0.526195
   validation loss: 24.162103
   validation acc: 0.375000
INFO:tensorflow:Froze 14 variables.
INFO:tensorflow:Converted 14 variables to const ops.
9811
   train loss: 14.627170
   train acc: 0.517923
   validation loss: 24.937206
   validation acc: 0.333333
INFO:tensorflow:Froze 14 variables.
INFO:tensorflow:Converted 14 variables to const ops.
9812
   train loss: 14.820483
   train acc: 0.512868
   validation loss: 25.643841
   validation acc: 0.328125
INFO:tensorflow:Froze 14 variables.
INFO:tensorflow:Converted 14 variables to const ops.


9849
   train loss: 14.521862
   train acc: 0.522059
   validation loss: 25.296550
   validation acc: 0.348958
INFO:tensorflow:Froze 14 variables.
INFO:tensorflow:Converted 14 variables to const ops.
9850
   train loss: 14.565441
   train acc: 0.513787
   validation loss: 24.902959
   validation acc: 0.338542
INFO:tensorflow:Froze 14 variables.
INFO:tensorflow:Converted 14 variables to const ops.
9851
   train loss: 14.125640
   train acc: 0.535846
   validation loss: 25.692259
   validation acc: 0.296875
INFO:tensorflow:Froze 14 variables.
INFO:tensorflow:Converted 14 variables to const ops.
9852
   train loss: 14.100045
   train acc: 0.537224
   validation loss: 25.781278
   validation acc: 0.328125
INFO:tensorflow:Froze 14 variables.
INFO:tensorflow:Converted 14 variables to const ops.
9853
   train loss: 14.157939
   train acc: 0.519761
   validation loss: 25.572635
   validation acc: 0.296875
INFO:tensorflow:Froze 14 variables.
INFO:tensorflow:Converted 14 variables to const ops.


9890
   train loss: 14.207853
   train acc: 0.532169
   validation loss: 24.457724
   validation acc: 0.343750
INFO:tensorflow:Froze 14 variables.
INFO:tensorflow:Converted 14 variables to const ops.
9891
   train loss: 14.249289
   train acc: 0.523897
   validation loss: 24.579411
   validation acc: 0.322917
INFO:tensorflow:Froze 14 variables.
INFO:tensorflow:Converted 14 variables to const ops.
9892
   train loss: 14.021961
   train acc: 0.528493
   validation loss: 24.225282
   validation acc: 0.348958
INFO:tensorflow:Froze 14 variables.
INFO:tensorflow:Converted 14 variables to const ops.
9893
   train loss: 14.560578
   train acc: 0.514246
   validation loss: 24.592789
   validation acc: 0.343750
INFO:tensorflow:Froze 14 variables.
INFO:tensorflow:Converted 14 variables to const ops.
9894
   train loss: 14.497832
   train acc: 0.517463
   validation loss: 23.553725
   validation acc: 0.307292
INFO:tensorflow:Froze 14 variables.
INFO:tensorflow:Converted 14 variables to const ops.


9931
   train loss: 14.233645
   train acc: 0.530790
   validation loss: 24.105567
   validation acc: 0.354167
INFO:tensorflow:Froze 14 variables.
INFO:tensorflow:Converted 14 variables to const ops.
9932
   train loss: 14.205001
   train acc: 0.530790
   validation loss: 24.405241
   validation acc: 0.343750
INFO:tensorflow:Froze 14 variables.
INFO:tensorflow:Converted 14 variables to const ops.
9933
   train loss: 14.064651
   train acc: 0.532169
   validation loss: 25.630969
   validation acc: 0.328125
INFO:tensorflow:Froze 14 variables.
INFO:tensorflow:Converted 14 variables to const ops.
9934
   train loss: 13.853005
   train acc: 0.545956
   validation loss: 25.288090
   validation acc: 0.333333
INFO:tensorflow:Froze 14 variables.
INFO:tensorflow:Converted 14 variables to const ops.
9935
   train loss: 13.929962
   train acc: 0.534926
   validation loss: 25.759598
   validation acc: 0.343750
INFO:tensorflow:Froze 14 variables.
INFO:tensorflow:Converted 14 variables to const ops.


9972
   train loss: 13.672584
   train acc: 0.552390
   validation loss: 25.187464
   validation acc: 0.354167
INFO:tensorflow:Froze 14 variables.
INFO:tensorflow:Converted 14 variables to const ops.
9973
   train loss: 14.080385
   train acc: 0.538143
   validation loss: 24.070707
   validation acc: 0.343750
INFO:tensorflow:Froze 14 variables.
INFO:tensorflow:Converted 14 variables to const ops.
9974
   train loss: 13.704830
   train acc: 0.548713
   validation loss: 24.076778
   validation acc: 0.338542
INFO:tensorflow:Froze 14 variables.
INFO:tensorflow:Converted 14 variables to const ops.
9975
   train loss: 13.825315
   train acc: 0.540441
   validation loss: 24.287804
   validation acc: 0.338542
INFO:tensorflow:Froze 14 variables.
INFO:tensorflow:Converted 14 variables to const ops.
9976
   train loss: 13.764339
   train acc: 0.539982
   validation loss: 24.469408
   validation acc: 0.333333
INFO:tensorflow:Froze 14 variables.
INFO:tensorflow:Converted 14 variables to const ops.
